In [250]:
import gradio as gr
import pandas as pd
import mlflow
import numpy as np
import shap
import mlflow.pyfunc
import sklearn
import pickle

In [228]:
mlflow.set_tracking_uri("http://localhost:5000")

In [249]:
model = load_model('rent-wue-selected-features-linear-reg')

2023/07/13 21:01:35 WARNING mlflow.pyfunc: Encountered an unexpected error (AttributeError("module 'configparser' has no attribute '__version__'")) while detecting model dependency mismatches. Set logging level to DEBUG to see the full traceback.
2023/07/13 21:01:35 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.8.16`, differs from the version of Python that is currently running, `Python 3.11.3`, and may be incompatible


In [256]:
X_train = pd.read_excel(r'C:\Users\mbauer2\workspace\Uni\enterprise-ai-project\ImmmoWelt_Price_Guide\data\X_train.xlsx')

# SHAP-KernelExplainer erstellen und analysieren
explainer = shap.KernelExplainer(model.predict, X_train)

Provided model function fails when applied to the provided data set.


MlflowException: Model is missing inputs ['LivingSpace', 'Rooms', 'altbau_(bis_1945)', 'balkon', 'barriefrei', 'dachgeschoss', 'einbaukueche', 'neubau', 'parkett', 'stellplatz', 'bad/wc_getrennt', 'personenaufzug', 'garten', 'garage', 'renoviert', 'terrasse', 'wanne', 'zentralheizung', 'abstellraum', 'ferne', 'fussbodenheizung', 'gartennutzung', 'kelleranteil', 'ZipCode_97070', 'ZipCode_97072', 'ZipCode_97074', 'ZipCode_97076', 'ZipCode_97078', 'ZipCode_97080', 'ZipCode_97082', 'ZipCode_97084', 'ZipCode_97204', 'ZipCode_97209', 'ZipCode_97218', 'ZipCode_97222', 'ZipCode_97228', 'ZipCode_97234', 'ZipCode_97236', 'ZipCode_97246', 'ZipCode_97249', 'ZipCode_97250', 'ZipCode_97261', 'ZipCode_97270', 'ZipCode_97288', 'ZipCode_97297', 'ZipCode_97299']. Note that there were extra inputs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54

In [230]:
def get_dataframe(
    feature_squrmeter,
    feature_zip,
    feature_rooms,
    features_altbau,
    feature_balkon,
    feature_ba,
    feature_dachgeschoss,
    feature_einbaukueche,
    feature_neubau,
    feature_parkett,
    feature_stellplatz,
    feature_badwc_getrennt,
    feature_personenaufzug,
    feature_garten,
    feature_garage,
    feature_renoviert,
    feature_terrasse,
    feature_wanne,
    feature_zentralheizung,
    feature_abstellraum,
    feature_fernwaerme,
    feature_fussbodenheitzung,
    feature_gartenmitbenutzung,
    feature_kellerabteil,
):
    data_list = (
        [feature_squrmeter]
        + [feature_rooms]
        + [features_altbau]
        + [feature_balkon]
        + [feature_ba]
        + [feature_dachgeschoss]
        + [feature_einbaukueche]
        + [feature_neubau]
        + [feature_parkett]
        + [feature_stellplatz]
        + [feature_badwc_getrennt]
        + [feature_personenaufzug]
        + [feature_garten]
        + [feature_garage]
        + [feature_renoviert]
        + [feature_terrasse]
        + [feature_wanne]
        + [feature_zentralheizung]
        + [feature_abstellraum]
        + [feature_fernwaerme]
        + [feature_fussbodenheitzung]
        + [feature_gartenmitbenutzung]
        + [feature_kellerabteil]
        + [int(0)] * 23
    )
    data = pd.DataFrame()
    data = pd.DataFrame(
        columns=[
            "LivingSpace",
            "Rooms",
            "altbau_(bis_1945)",
            "balkon",
            "barriefrei",
            "dachgeschoss",
            "einbaukueche",
            "neubau",
            "parkett",
            "stellplatz",
            "bad/wc_getrennt",
            "personenaufzug",
            "garten",
            "garage",
            "renoviert",
            "terrasse",
            "wanne",
            "zentralheizung",
            "abstellraum",
            "ferne",
            "fussbodenheizung",
            "gartennutzung",
            "kelleranteil",
            "ZipCode_97070",
            "ZipCode_97072",
            "ZipCode_97074",
            "ZipCode_97076",
            "ZipCode_97078",
            "ZipCode_97080",
            "ZipCode_97082",
            "ZipCode_97084",
            "ZipCode_97204",
            "ZipCode_97209",
            "ZipCode_97218",
            "ZipCode_97222",
            "ZipCode_97228",
            "ZipCode_97234",
            "ZipCode_97236",
            "ZipCode_97246",
            "ZipCode_97249",
            "ZipCode_97250",
            "ZipCode_97261",
            "ZipCode_97270",
            "ZipCode_97288",
            "ZipCode_97297",
            "ZipCode_97299",
        ]
    )

    data.loc[len(data)] = data_list
    for index, row in data.iterrows():
        for columns in data.columns:
            if feature_zip[:5] in columns:
                data.loc[index, columns] = int(1)
    # wenn false dann 1 wenn true dann 1 im datframe erstzen
    data = data.replace(False, 0)
    data = data.replace(True, 1)
    # wandele alle werte der zipcode spalten in int32 um
    data = data.astype({"ZipCode_97070": np.int32})
    data = data.astype({"ZipCode_97072": np.int32})
    data = data.astype({"ZipCode_97074": np.int32})
    data = data.astype({"ZipCode_97076": np.int32})
    data = data.astype({"ZipCode_97078": np.int32})
    data = data.astype({"ZipCode_97080": np.int32})
    data = data.astype({"ZipCode_97082": np.int32})
    data = data.astype({"ZipCode_97084": np.int32})
    data = data.astype({"ZipCode_97204": np.int32})
    data = data.astype({"ZipCode_97209": np.int32})
    data = data.astype({"ZipCode_97218": np.int32})
    data = data.astype({"ZipCode_97222": np.int32})
    data = data.astype({"ZipCode_97228": np.int32})
    data = data.astype({"ZipCode_97234": np.int32})
    data = data.astype({"ZipCode_97236": np.int32})
    data = data.astype({"ZipCode_97246": np.int32})
    data = data.astype({"ZipCode_97249": np.int32})
    data = data.astype({"ZipCode_97250": np.int32})
    data = data.astype({"ZipCode_97261": np.int32})
    data = data.astype({"ZipCode_97270": np.int32})
    data = data.astype({"ZipCode_97288": np.int32})
    data = data.astype({"ZipCode_97297": np.int32})
    data = data.astype({"ZipCode_97299": np.int32})

    # printe das ganze dataframe das alle reihen und spalten zu sehen sind
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        print(data)
    data_model = data
    preds = model.predict(data_model)
    preds = int(preds)
    print(preds)
    print(type(preds))
    html_response = f"""
        <html>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin-top: 200px;
                }}
                
                h1 {{
                    color: #333;
                    font-size: 24px;
                    text-align: center;
                }}
                
                p {{
                    color: #777;
                    font-size: 22px;
                    text-align: center;
                }}
            </style>
        </head>
        <body>
            <h1>Der vorgeschlagene Preis für die Immobilie beträgt:</h1>
            <p>{preds} €</p>
        </body>
        </html>
    """
    return html_response

In [231]:
def load_model(model_name):
    model_name = 'rent-wue-selected-features-linear-reg'
    model_version = 1
    model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
    return model

In [232]:
with gr.Blocks() as demo:
    gr.Markdown(
    """
    # Geben Sie hier an, welche Merkmale Ihre Immobilie besitzt 🏠:
    """
    )
    with gr.Row():
        feature_zip = gr.Dropdown(
            label="Postleitzahl",
            choices=[
                "97070 Würzburg-Altstadt",
                "97072 Würzburg-Sanderau",
                "97074 Würzburg-Frauenland",
                "97076 Würzburg-Lengfeld",
                "97078 Würzburg-Lindleinsmühle",
                "97080 Würzburg-Dürrbachtal",
                "97082 Würzburg-Steinbachtal",
                "97084 Würzburg-Heuchelhof",
                "97204 Höchberg",
                "97209 Veitshöchheim",
                "97218 Gerbrunn",
                "97222 Rimpar",
                "97228 Rottendorf",
                "97234 Reichenberg",
                "97236 Randersacker",
                "97246 Eibelstadt",
                "97249 Eisingen",
                "97250 Erlabrunn",
                "97261 Güntersleben",
                "97270 Kist",
                "97288 Theilheim",
                "97297 Waldbüttelbrunn",
                "97299 Zell am Main",
            ],
        )
    with gr.Row():
        feature_squrmeter = gr.Number(label="Wohnfläche [qm]")
    with gr.Row():
        feature_rooms = gr.Number(label="Anzahl der Zimmer")
    with gr.Row():
        features_altbau = gr.Checkbox(label="Altbau (bis 1945)")
        feature_balkon = gr.Checkbox(label="Balkon")
        feature_ba = gr.Checkbox(label="Barrierefrei")
        feature_dachgeschoss = gr.Checkbox(label="Dachgeschoss")
        feature_einbaukueche = gr.Checkbox(label="Einbauküche")
        feature_neubau = gr.Checkbox(label="Neubau")
        feature_parkett = gr.Checkbox(label="Parkett")
        feature_stellplatz = gr.Checkbox(label="Stellplatz")
        feature_badwc_getrennt = gr.Checkbox(label="Bad/WC getrennt")
        feature_personenaufzug = gr.Checkbox(label="Personenaufzug")
        feature_garten = gr.Checkbox(label="Garten")
        feature_garage = gr.Checkbox(label="Garage")
        feature_renoviert = gr.Checkbox(label="Renoviert")
        feature_terrasse = gr.Checkbox(label="Terrasse")
        feature_wanne = gr.Checkbox(label="Wanne")
        feature_zentralheizung = gr.Checkbox(label="Zentralheizung")
        feature_abstellraum = gr.Checkbox(label="Abstellraum")
        feature_fernwaerme = gr.Checkbox(label="Fernwärme")
        feature_fussbodenheitzung = gr.Checkbox(label="Fußbodenheizung")
        feature_gartenmitbenutzung = gr.Checkbox(label="Gartenmitbenutzung")
        feature_kellerabteil = gr.Checkbox(label="Kellerabteil")
    btn = gr.Button("Berechne deinen Preis")
    preds = gr.Number(label="Dein Preis", visible=False)

    output_html = gr.HTML(label="Dein Preis")
    btn.click(
        fn=get_dataframe,
        inputs=[
            feature_squrmeter,
            feature_zip,
            feature_rooms,
            features_altbau,
            feature_balkon,
            feature_ba,
            feature_dachgeschoss,
            feature_einbaukueche,
            feature_neubau,
            feature_parkett,
            feature_stellplatz,
            feature_badwc_getrennt,
            feature_personenaufzug,
            feature_garten,
            feature_garage,
            feature_renoviert,
            feature_terrasse,
            feature_wanne,
            feature_zentralheizung,
            feature_abstellraum,
            feature_fernwaerme,
            feature_fussbodenheitzung,
            feature_gartenmitbenutzung,
            feature_kellerabteil,
        ],
        #gebe mir den output der funktion in einem number gradio widget aus

        outputs=output_html
    )

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


   LivingSpace  Rooms  altbau_(bis_1945)  balkon  barriefrei  dachgeschoss  \
0        100.0    5.0                  1       0           0             0   

   einbaukueche  neubau  parkett  stellplatz  bad/wc_getrennt  personenaufzug  \
0             0       0        0           0                0               0   

   garten  garage  renoviert  terrasse  wanne  zentralheizung  abstellraum  \
0       0       0          0         0      0               0            0   

   ferne  fussbodenheizung  gartennutzung  kelleranteil  ZipCode_97070  \
0      0                 0              0             0              1   

   ZipCode_97072  ZipCode_97074  ZipCode_97076  ZipCode_97078  ZipCode_97080  \
0              0              0              0              0              0   

   ZipCode_97082  ZipCode_97084  ZipCode_97204  ZipCode_97209  ZipCode_97218  \
0              0              0              0              0              0   

   ZipCode_97222  ZipCode_97228  ZipCode_97234  ZipC